In [1]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou

Dependencies:
torch: 0.4
torchvision
matplotlib
"""
# library
# standard library
import os

# third-party library
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

from torchvision.datasets import ImageFolder
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision import datasets

import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
def load_images(image_size=(28,28), batch_size=64, root="../datasets/MainImageFolder"):

    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_set = datasets.ImageFolder(root=root, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    return train_loader 

In [3]:
train_data = load_images(root='./state-farm-distracted-driver-detection/imgs/train/')

In [12]:
# Insipired on LeNet, first proposed in: 
# https://ieeexplore.ieee.org/abstract/document/726791
# PDF: http://www.cs.virginia.edu/~vicente/deeplearning/readings/lecun1998.pdf
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # Convolutional layers.
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        # Linear layers.
        self.fc1 = nn.Linear(64*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Conv1 + ReLU + MaxPooling.
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        
        # Conv2 + ReLU + MaPooling.
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        
        # This flattens the output of the previous layer into a vector.
        out = out.view(out.size(0), -1) 
        
        # Linear layer + ReLU.
        out = F.relu(self.fc1(out))
        # Linear layer + ReLU.
        out = F.relu(self.fc2(out))
        # A final linear layer at the end.
        out = self.fc3(out)
       
        # We will not add Softmax here because nn.CrossEntropy does it.
        # Read the documentation for nn.CrossEntropy.
        return out

model = LeNet()

model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
correct = 0.0
cum_loss = 0.0

learningRate = 5e-2
optimizer = optim.SGD(model.parameters(), lr = learningRate, 
                      momentum = 0.9, weight_decay = 1e-4)
N = 5
# log accuracies and losses.
train_accuracies = []; val_accuracies = []
train_losses = []; val_losses = []
batchSize=64

In [13]:
for epoch in range(0, N):
        correct = 0.0
        cum_loss = 0.0

      # Make a pass over the training data.
        model.train()
        for (i, (inputs, labels)) in enumerate(train_data):
            inputs = inputs.cuda()
#             print(inputs.shape)
            labels = labels.cuda()

            # Forward pass. (Prediction stage)
            scores = model(inputs)
            loss = loss_fn(scores, labels)

            # Count how many correct in this batch.
            max_scores, max_labels = scores.max(1)
            correct += (max_labels == labels).sum().item()
            cum_loss += loss.item()

            # Zero the gradients in the network.
            optimizer.zero_grad()

            # Backward pass. (Gradient computation stage)
            loss.backward()

            # Parameter updates (SGD step) -- if done with torch.optim!
            optimizer.step()

            # Parameter updates (SGD step) -- if done manually!
            # for param in model.parameters():
            #   param.data.add_(-learningRate, param.grad)

            # Logging the current results on training.
            if (i + 1) % 100 == 0:
                print('Train-epoch %d. Iteration %05d, Avg-Loss: %.4f, Accuracy: %.4f' % (epoch, i + 1, cum_loss / (i + 1), correct / ((i + 1) * batchSize)))

            train_accuracies.append(correct / len(train_data))
            train_losses.append(cum_loss / (i + 1))   

        #     # Make a pass over the validation data.
        #     correct = 0.0
        #     cum_loss = 0.0
        #     model.eval()
        #     for (i, (inputs, labels)) in enumerate(val_loader):
        #         inputs = inputs.cuda()
        #         labels = labels.cuda()

        #         # Forward pass. (Prediction stage)
        #         scores = model(inputs)
        #         cum_loss += loss_fn(scores, labels).item()

        #         # Count how many correct in this batch.
        #         max_scores, max_labels = scores.max(1)
        #         correct += (max_labels == labels).sum().item()

        #     val_accuracies.append(correct / len(valset))
        #     val_losses.append(cum_loss / (i + 1))

        #     # Logging the current results on validation.
        #     print('Validation-epoch %d. Avg-Loss: %.4f, Accuracy: %.4f' % 
        #         (epoch, cum_loss / (i + 1), correct / len(valset)))

Train-epoch 0. Iteration 00100, Avg-Loss: 2.2013, Accuracy: 0.1527
Train-epoch 0. Iteration 00200, Avg-Loss: 1.7914, Accuracy: 0.3112
Train-epoch 0. Iteration 00300, Avg-Loss: 1.4638, Accuracy: 0.4489
Train-epoch 1. Iteration 00100, Avg-Loss: 0.3547, Accuracy: 0.8886
Train-epoch 1. Iteration 00200, Avg-Loss: 0.3191, Accuracy: 0.9009
Train-epoch 1. Iteration 00300, Avg-Loss: 0.2820, Accuracy: 0.9135
Train-epoch 2. Iteration 00100, Avg-Loss: 0.1756, Accuracy: 0.9461
Train-epoch 2. Iteration 00200, Avg-Loss: 0.1690, Accuracy: 0.9495
Train-epoch 2. Iteration 00300, Avg-Loss: 0.1496, Accuracy: 0.9559
Train-epoch 3. Iteration 00100, Avg-Loss: 0.0870, Accuracy: 0.9747
Train-epoch 3. Iteration 00200, Avg-Loss: 0.0840, Accuracy: 0.9741
Train-epoch 3. Iteration 00300, Avg-Loss: 0.0908, Accuracy: 0.9726
Train-epoch 4. Iteration 00100, Avg-Loss: 0.0937, Accuracy: 0.9694
Train-epoch 4. Iteration 00200, Avg-Loss: 0.0860, Accuracy: 0.9729
Train-epoch 4. Iteration 00300, Avg-Loss: 0.0824, Accuracy: 0.